# Comparing primal and dual simplex for the same LP

In this example, we will show how the selection between primal and dual simplex has impacts on performance. Solvers typically decide whether to apply primal or dual simplex by trying them concurrently or picking one as default. 

In general, the are no set rules that can guarantee that one will perform better than the other. Some elements might serve as indication, such as having a significantly difference in the number of variables and constraints, or the occurence of degeneracy in the primal or dual. As a general guideline from numerical experience, dual simplex tends to perform better than primal simplex, but there are several cases where primal simplex can be shown to be best performing.

We will use Clp to show the difference betweeen primal and dual simplex. Clp is an open-source solver that has impelementations of both primal and dual simplex methods.

Links for documentation: 
https://projects.coin-or.org/Clp
https://github.com/JuliaOpt/Clp.jl

Clp class documentation: https://www.coin-or.org/Doxygen/Clp/annotated.html

In [2]:
#import Pkg; Pkg.add("Clp")
using Clp
using JuMP
using LinearAlgebra

In [3]:
function linprog(c, A, b, Solver)
# Emulates Matlab's linprog, in the format min c'x : Ax >= b, x >= 0.  
# The parameter Solver controls what method is to be used: 0 - dual simplex or 1 - primal simplex. See here for full list: https://www.coin-or.org/Doxygen/Clp/classClpSolve.html#a3e2f401fca0ad2ea77c7486aa4d4c3f2    
    m,n = size(A) # m constraints, n variables

    lp = Model(with_optimizer(Clp.Optimizer, LogLevel = 1, SolveType = Solver, PresolveType = 1))

    @variable(lp, x[1:n] >= 0)
    @constraint(lp, cons, A*x .>= b)
    @objective(lp, Min, sum(c[i]*x[i] for i=1:n))

    optimize!(lp)
end

linprog (generic function with 1 method)

In [4]:
# Generating a problem with m constraints and n variables. The parameter sparsity enforces 
# an artifical sparcity in matrix A of roughly the amount of the parameter.

m = 1000
n = 1000
sparsity = 0.5

A = rand(m,n)                 # A is comprised by elements in [0,1].
for i = 1:m
    for j = 1:n
        if A[i,j] <= sparsity # Enforces sparsity of matrix A.
            A[i,j] = 0
        end    
    end
end

b = 1 .+ rand(m)              # b is comprised by elements [1,2]
c= rand(n);

We now solve the same problem with primal simplex first and then dual simplex. 

In [5]:
println("Solving with primal simplex")
linprog(c, A, b, 1) # 1 - primal simplex

Solving with primal simplex
Clp0006I 0  Obj 0 Primal inf 1507.412 (1000) Dual inf 3.7439393e+15 (1000)
Clp0006I 31  Obj 4.092714 Dual inf 3646.8387 (591)
Clp0006I 86  Obj 1.1098316 Dual inf 1149.3654 (415)
Clp0006I 148  Obj 0.5622999 Dual inf 384.08204 (415)
Clp0006I 230  Obj 0.341216 Dual inf 251.87281 (367)
Clp0006I 296  Obj 0.21254743 Dual inf 76.240034 (206)
Clp0006I 362  Obj 0.12941166 Dual inf 18.313635 (136)
Clp0006I 407  Obj 0.1009439 Dual inf 0.0042360155 (4)
Clp0006I 411  Obj 0.10060524
Clp0000I Optimal - objective value 0.10060524
Clp0032I Optimal objective 0.1006052358 - 411 iterations time 0.312


In [6]:
println("Solving with dual simplex")
linprog(c, A, b, 0) # 0 - dual simplex

Solving with dual simplex
Clp0006I 0  Obj 0 Primal inf 1507.412 (1000)
Clp0006I 47  Obj 0.10060524
Clp0000I Optimal - objective value 0.10060524
Clp0032I Optimal objective 0.1006052358 - 47 iterations time 0.032


Notice the last lines of the logs for the two calls, solving first with primal simplex, and then with dual simplex. How they compare in terms of time and number of iterations? For a problem like the one being generated, typically dual simplex will perform better, since a dual feasible basis is easily available in this case. Notice that solving it with primal simplex there is a report of primal infeasibility at first, meaning that a feasibility phase is employed. Later, we only see dual infeasibility, which, as we seen in class, is equivalent to "lack of optimality". When the problem becomes dual feasible, it means it found a primal optimal solution.    